Jose Valenti's 2-D testing notebook with SEA swimming kernel added

In [ ]:
%matplotlib inline
from parcels import Field, FieldSet, ParticleSet,Variable, JITParticle, ErrorCode
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc, animation
import xarray as xr
from datetime import datetime
import os

from IPython.display import Image
rc('animation', html='html5')

In [ ]:
def make_prefix(date, res='h'):
    """Construct path prefix for local SalishSeaCast results given date object and paths dict
    e.g., /results2/SalishSea/nowcast-green.201905/daymonthyear/SalishSea_1h_yyyymmdd_yyyymmdd
    """
    path = '/results2/SalishSea/nowcast-green.202111/'
    datestr = '_'.join(np.repeat(date.strftime('%Y%m%d'), 2))
    folder = date.strftime("%d%b%y").lower()
    prefix = os.path.join(path, f'{folder}/SalishSea_1{res}_{datestr}')
    
    return prefix

In [ ]:
# Pick hour,day and location.
hour = 0 # ? Start hour of day in 24 hour format?
path_NEMO = make_prefix(datetime(2018, 8, 1)) # Choose start year, month, day
lati = 49.224563 # Choose coords
loni = -123.605357
jjii = xr.open_dataset('/home/sallen/MEOPAR/grid/grid_from_lat_lon_mask999.nc') # ? Does this retrieve the SalishSeaCast coordinate grid?
j = [jjii.jj.sel(lats=lati, lons=loni, method='nearest').item()][0] # ? What do j and i represent, what is the difference between them, and why choose that notation?
i = [jjii.ii.sel(lats=lati, lons=loni, method='nearest').item()][0]

# Specify some simulation parameters
T = 5e3 * 100 #s (run time)
dt = 5 #s (timestep)
N = 10e3 #number of particles
outputdt = 500 #s (how often do you want output?)
outputpath = '/ocean/mattmiller/MOAD/results/parcels/test/Outputmix_sink_and_swim.zarr'

In [ ]:
def DeleteParticle(particle, fieldset, time):
    """Delete particle from OceanParcels simulation to avoid run failure"""
    print(f'Particle {particle.id} lost !! [{particle.time}, {particle.depth}, {particle.lat}, {particle.lon}]')
    particle.delete()

In [ ]:
# Load data from NEMO
Wdat = xr.open_dataset(path_NEMO+'_grid_W.nc', decode_times=False) # Opens the W grid data from SSC
mbathy = xr.open_dataset('/ocean/jvalenti/MOAD/grid/mesh_mask202108.nc').mbathy[0,j,i].item() # ? Why is the value of this 37? What does it represent?
depth = np.array(Wdat.depthw[:mbathy+1]).astype('float32') # ? Are these depth values the vertical grid spacing?
Kz_col = np.array(Wdat.vert_eddy_diff[hour,:mbathy+1,j,i]).astype('float32') # Assigns the vertical_eddy_diffusivity values to "Kz_col"
#Kz_col[0]=Kz_col[1] # ? Can I delete this line?

In [ ]:
depth

In [ ]:
depth2 = np.arange(depth[0],depth[-1],1)
Kz2 = np.interp(depth2, depth, Kz_col)
plt.plot(Kz_col,-depth)
Kzp = np.gradient(Kz2, depth2[1]-depth2[0])
Kzp2 = np.gradient(Kzp, depth2[1]-depth2[0])
dtmax = np.min(1/abs(Kzp2))
print(f'Dt should be << {dtmax}')

In [ ]:
# Here you can define as many particle variables as you want specifying the initial values. In this case, I use the variable to set a constant.
class MPParticle(JITParticle):
    Kh = Variable('Kh', initial =  0.0) # If you want horizontal mixing activated change value here
    bath = Variable('bath', initial= depth[-1])

In [ ]:
# Mixing Kernel ? I'm not sure what most of these lines do. Could you annotate/explain?
def tu_mix(particle,fieldset,time): 
    bath = particle.bath #109.73707
    if particle.depth+1 > bath: #Forward euler
        Kzdz = (fieldset.Kz[time,particle.depth,particle.lat, particle.lon]-fieldset.Kz[time,particle.depth-1,particle.lat, particle.lon]) #forward difference 
    else:
        Kzdz = (fieldset.Kz[time,particle.depth+1,particle.lat, particle.lon]-fieldset.Kz[time,particle.depth,particle.lat, particle.lon]) #backward difference 
    dgrad = Kzdz * particle.dt 
    if particle.depth+0.5*dgrad > 0 and particle.depth+0.5*dgrad < bath:
        kk = fieldset.Kz[time,particle.depth+0.5*dgrad,particle.lat, particle.lon] #Vertical diffusivity SSC  
    else:
        kk = fieldset.Kz[time, bath,particle.lat, particle.lon] #Vertical diffusivity SSC 
    Kx = particle.Kh
    Rr = ParcelsRandom.uniform(-1, 1)
    Rr2 = ParcelsRandom.uniform(-1, 1)
    d_random = sqrt(3*2*kk*particle.dt) * Rr
    d_randomx = sqrt(3*2*Kx*particle.dt) * Rr2
    Dlayerz = 0.5*sqrt(kk * particle.dt) #1m mixing layer
    Dlayerx = 0.5*sqrt(Kx * particle.dt)
    if particle.lon + d_randomx < 0:
        particle.lon = Dlayerx * ParcelsRandom.uniform(0, 1)
    elif particle.lon + d_randomx > 2e3:
        particle.lon = 2e3 - Dlayerx * ParcelsRandom.uniform(0, 1)
    else: 
        particle.lon=particle.lon + d_randomx
    if d_random + dgrad +particle.depth > bath: #randomly in boundary mixed layer
        particle.depth = bath - Dlayerz * ParcelsRandom.uniform(0, 1)
    elif d_random +particle.depth +dgrad < 0:
            particle.depth = Dlayerz * ParcelsRandom.uniform(0, 1) #Well mixed boundary layer
    else:
        particle.depth += d_random + dgrad 

In [ ]:
#Swimming Kernel
def tu_swim(particle, fieldset, time):
    mbath = particle.bath
    sink = 20e-3
    swim = 32.5e-3
    dt = 200 # how often they change direction
    kappa = swim**2 * dt # Double asterisk means exponential
    eff_swim = sqrt(kappa / particle.dt)

    upward = 1.05   # tendency to swim more upward than sink
    zswim = (- particle.dt * (upward-1)*sink + particle.dt * 
             (eff_swim - upward*sink) * math.cos(ParcelsRandom.uniform(0, 2* math.pi))) # ? I don't understand the math behind this
    if particle.depth + zswim < 0:
        particle.depth = 0
    elif particle.depth + zswim > mbath:
        particle.depth = mbath
    else:
        particle.depth += zswim

In [ ]:
#Define domain, velocity fields and Kz 
dim = 100
dep = len(depth)
lon = np.linspace(0., 2e3, dim, dtype=np.float32)

#Build fieldsets for OP
U = Field('U', np.zeros((dep, dim), dtype=np.float32), lon=lon, depth=depth) # ? Is this just setting U and V to 0 because we are only using W data?
V = Field('V', np.zeros((dep, dim), dtype=np.float32), lon=lon, depth=depth)
Kz_data = np.zeros((dep, dim), dtype=np.float32)
for i in range(dim):
    Kz_data[:,i]=Kz_col   
Kz = Field('Kz', Kz_data, grid=U.grid)
fieldset = FieldSet(U,V)
fieldset.add_field(Kz)

In [ ]:
#Start with random distribution
lon_g = np.random.uniform(low=lon[0], high=lon[-1], size=(int(N),))
depth_g = np.random.uniform(low=depth[0], high=depth[-1], size=(int(N),))
lat_g = np.zeros(int(N),)

In [ ]:
def run_turb_test(lon_g,depth_g):
    pset = ParticleSet(fieldset, pclass=MPParticle, lon=lon_g, depth=depth_g,lat = lat_g)
    output_file = pset.ParticleFile(name=outputpath, outputdt=outputdt)
    KE = tu_swim + pset.Kernel(tu_mix) #if you want to add more kernels use:  KE = tu_mix + NAME_KERNEL1 + ...
    pset.execute(KE , runtime=T, dt=dt, output_file=output_file,recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

In [ ]:
run_turb_test(lon_g,depth_g)

Load data

In [ ]:
dat = xr.load_dataset('/ocean/mattmiller/MOAD/results/parcels/test/Outputmix_sink_and_swim.zarr')
#dat = xr.load_dataset('/ocean/mattmiller/MOAD/results/parcels/test/Outputmix_no_sink.zarr')
#dat = xr.load_dataset('/ocean/mattmiller/MOAD/results/parcels/test/Outputmix_passive_particles.zarr')

Plots

In [ ]:
binwidths = [10, 15, 25, 25, 25, 50, 100]
counts, bins = np.histogram(dat.z[:, 0], bins = [0, 10, 25, 50, 75, 100, 150, 250])
plt.stairs((counts/binwidths)/sum(counts/binwidths), bins)
plt.xticks([0, 10, 25, 50, 75, 100, 150, 250])
plt.yticks(np.arange(0, 1.1, 0.1))
plt.xlabel('Depth (m)')
plt.ylabel('Relative Particle Abundance')
counts[0]

In [ ]:
# No sinking
binwidths = [10, 15, 25, 25, 25, 50, 100]
counts, bins = np.histogram(dat.z[:, -1], bins = [0, 10, 25, 50, 75, 100, 150, 250])
plt.stairs((counts/binwidths)/sum(counts/binwidths), bins)
plt.xticks([0, 10, 25, 50, 75, 100, 150, 250])
plt.yticks(np.arange(0, 1.1, 0.1))
plt.xlabel('Depth (m)')
plt.ylabel('Relative Particle Abundance')
counts[0]

In [ ]:
# sink_and_swim
binwidths = [10, 15, 25, 25, 25, 50, 100]
counts, bins = np.histogram(dat.z[:, -1], bins = [0, 10, 25, 50, 75, 100, 150, 250])
plt.stairs((counts/binwidths)/sum(counts/binwidths), bins)
plt.xticks([0, 10, 25, 50, 75, 100, 150, 250])
plt.yticks(np.arange(0, 1.1, 0.1))
plt.xlabel('Depth (m)')
plt.ylabel('Relative Particle Abundance')
counts[0]

In [ ]:
# sink_and_swim - initial timestep
counts, bins = np.histogram(dat.z[:, 0], bins=[0, 10, 25, 50, 75, 100, 150, 250]) # Calculate the counts and bins for the histogram
bin_centers = [0,  10,  25,  50,  75, 100, 150] # Calculate the bin centers (for placing bins on the y-axis
binwidths = np.diff(bins) #[10, 15, 25, 25, 25, 50, 100] # Define the binwidths
relative_abundance = (counts/binwidths)/sum(counts/binwidths) # Calculate the relative particle abundance
vertical_dist_medians = [0.350311667, 0.239004743, 0.113561039, 0.025229733, 0.007112793, 0.002542321, 0.003152159]
tstamp = dat.time[0, 0].values.astype('timedelta64[ns]').astype(datetime)

fig, (ax1, ax2) = plt.subplots(1,2,figsize=(10,8))
ax1.scatter(dat.lon[:,0],dat.z[:,0],s=3,c ='tab:blue')
ax1.set_yticks(bins, bins) # Set the tick positions and labels for the y-axis
ax1.set_ylim(0, 250)
ax1.invert_yaxis()
ax1.xaxis.tick_top()
ax1.xaxis.set_label_position('top')
ax1.set_xlabel('Lon')
ax1.set_ylabel('Depth (m)')
ax1.text(x = 0.02, y = -0.03, s = tstamp, transform=ax1.transAxes)
ax2.barh(bin_centers, relative_abundance, label = 'particle dist', height=binwidths, align='edge', color='tab:blue') # Create the histogram with bins on the y-axis and counts on the x-axis
ax2.barh(bin_centers, vertical_dist_medians, label = 'reference dist', height=binwidths, align='edge', color='none', edgecolor ='r') # Plot reference vertical distribution
ax2.set_yticks(bins, bins) # Set the tick positions and labels for the y-axis
ax2.set_ylim(0, 250)
ax2.invert_yaxis()
ax2.set_xticks(np.arange(0, 1.1, 0.1)) # Set the tick positions and labels for the x-axis
ax2.xaxis.tick_top()
ax2.xaxis.set_label_position('top')
ax2.set_xlabel('Relative Particle Abundance') # Add labels and title
ax2.set_ylabel('Depth (m)')
ax2.text(x = 0.02, y = -0.03, s = tstamp, transform=ax2.transAxes)
ax2.legend()
plt.suptitle('Sink and Swim')

In [ ]:
# sink_and_swim - final timestep
counts, bins = np.histogram(dat.z[:, -1], bins=[0, 10, 25, 50, 75, 100, 150, 250]) # Calculate the counts and bins for the histogram
bin_centers = [0,  10,  25,  50,  75, 100, 150] # Calculate the bin centers (for placing bins on the y-axis
binwidths = np.diff(bins) #[10, 15, 25, 25, 25, 50, 100] # Define the binwidths
relative_abundance = (counts/binwidths)/sum(counts/binwidths) # Calculate the relative particle abundance
vertical_dist_medians = [0.350311667, 0.239004743, 0.113561039, 0.025229733, 0.007112793, 0.002542321, 0.003152159]
tstamp = dat.time[0, -1].values.astype('timedelta64[ns]').astype(datetime)

fig, (ax1, ax2) = plt.subplots(1,2,figsize=(10,8))
ax1.scatter(dat.lon[:,-1],dat.z[:,-1],s=3,c ='tab:blue')
ax1.set_yticks(bins, bins) # Set the tick positions and labels for the y-axis
ax1.set_ylim(0, 250)
ax1.invert_yaxis()
ax1.xaxis.tick_top()
ax1.xaxis.set_label_position('top')
ax1.set_xlabel('Lon')
ax1.set_ylabel('Depth (m)')
ax1.text(x = 0.02, y = -0.03, s = tstamp, transform=ax1.transAxes)
ax2.barh(bin_centers, relative_abundance, label = 'particle dist', height=binwidths, align='edge', color='tab:blue') # Create the histogram with bins on the y-axis and counts on the x-axis
ax2.barh(bin_centers, vertical_dist_medians, label = 'reference dist', height=binwidths, align='edge', color='none', edgecolor ='r') # Plot reference vertical distribution
ax2.set_yticks(bins, bins) # Set the tick positions and labels for the y-axis
ax2.set_ylim(0, 250)
ax2.invert_yaxis()
ax2.set_xticks(np.arange(0, 1.1, 0.1)) # Set the tick positions and labels for the x-axis
ax2.xaxis.tick_top()
ax2.xaxis.set_label_position('top')
ax2.set_xlabel('Relative Particle Abundance') # Add labels and title
ax2.set_ylabel('Depth (m)')
ax2.text(x = 0.02, y = -0.03, s = tstamp, transform=ax2.transAxes)
ax2.legend()
plt.suptitle('Sink and Swim')

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.hist(dat.z[:, 0], bins = [0, 10, 25, 50, 75, 100, 150, 250], alpha=0.2, label='Start');
ax.hist(dat.z[:, 250], bins = [0, 10, 25, 50, 75, 100, 150, 250], alpha=0.2, label='1.2e5s');
ax.hist(dat.z[:, 500], bins = [0, 10, 25, 50, 75, 100, 150, 250], alpha=0.2, label='2.5e5s');
ax.hist(dat.z[:, 750], bins = [0, 10, 25, 50, 75, 100, 150, 250], alpha=0.2, label='3.8e5s');
ax.hist(dat.z[:, 1000], bins = [0, 10, 25, 50, 75, 100, 150, 250], alpha=0.2, label='5e5s');
ax.legend();
ax.set_xlabel('Depth (m)')
ax.set_ylabel('Counts, 10000 total');

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.hist(dat.z[:, 0], alpha=0.2, label='Start');
ax.hist(dat.z[:, 250], alpha=0.2, label='1.2e5s');
ax.hist(dat.z[:, 500], alpha=0.2, label='2.5e5s');
ax.hist(dat.z[:, 750], alpha=0.2, label='3.8e5s');
ax.hist(dat.z[:, 1000], alpha=0.2, label='5e5s');
ax.legend();
ax.set_xlabel('Depth (m)')
ax.set_ylabel('Counts, 10000 total');

View data

In [ ]:
print(dat)

Animations

In [ ]:
def anim(dat,fps=1):  
    fig, axs = plt.subplots(1,1,figsize=(5,10))
    axs.invert_yaxis()
    axs.set_title('Sink and Swim')
    axs.set_ylabel('Depth (m)')

    def update(frame):           
        global ss        
        for scat in ss:              
            scat.remove()            
        ss =[]                    
        ss.append(axs.scatter(dat.lon[:,frame],dat.z[:,frame],s=3,c ='tab:blue'))                     
        print(f'{frame/len(dat):.2f}% completed')           
        return ss
    return animation.FuncAnimation(fig, update, frames=np.arange(0,1000,fps), interval=1)

In [ ]:
ss =[]
ani = anim(dat,fps=1)
f = r'/ocean/mattmiller/MOAD/results/parcels/test/_1.gif'
FFwriter = animation.FFMpegWriter()
ani.save(f, writer = FFwriter)

In [ ]:
from IPython.display import Image
with open(f,'rb') as anim:
     display(Image(anim.read()))

In [ ]:
fig, axs=plt.subplots(1,1,figsize=(5,10))
axs.scatter(dat.lon[:,-1],dat.z[:,-1],s=3) # s = point size
#axs.plot(box[0],box[1],c='grey',linewidth=3)
axs.invert_yaxis()
axs.set_title('Sink and Swim')
axs.set_ylabel('Depth (m)')

In [ ]:

def anim(dat,fps=1):  
    fig, axs = plt.subplots(1,1,figsize=(5,10))
    binwidths = [10, 15, 25, 25, 25, 50, 100] # Define the binwidths
    bin_centers = [0,  10,  25,  50,  75, 100, 150]
    vertical_dist_medians = [0.350311667, 0.239004743, 0.113561039, 0.025229733, 0.007112793, 0.002542321, 0.003152159]
    axs.set_ylim(0, 250)
    axs.invert_yaxis() # flip y-axis
    axs.set_yticks([0, 10, 25, 50, 75, 100, 150, 250]) # Set the tick positions and labels for the y-axis
    axs.set_xticks(np.arange(0, 1.1, 0.1))
    axs.xaxis.set_ticks_position('top') # Move the x-axis to the top
    axs.xaxis.set_label_position('top')
    axs.set_xlabel('Relative Particle Abundance') # Add labels and title
    axs.set_ylabel('Depth (m)')
    plt.suptitle('Sink and Swim')

    def update(frame):           
        global ss        
        for scat in ss:              
            scat.remove()            
        ss =[]        
        tstamp = dat.time[0, frame].values.astype('timedelta64[ns]').astype(datetime)
        counts, bins = np.histogram(dat.z[:, frame], bins=[0, 10, 25, 50, 75, 100, 150, 250]) # Calculate the counts and bins for the histogram            
        relative_abundance = (counts/binwidths)/sum(counts/binwidths) # Calculate the relative particle abundance
        ss.append(axs.barh(bin_centers, relative_abundance, label = 'particle dist', height=np.diff(bins), align='edge', color = 'tab:blue'))                             
        ss.append(axs.barh(bin_centers, vertical_dist_medians, label = 'reference dist', height=np.diff(bins), align='edge', color = 'none', edgecolor = 'r'))
        ss.append(axs.text(x = 0.02, y = -0.03, s = tstamp, transform=axs.transAxes))
        ss.append(axs.legend())
        print(f'{frame/len(dat.z.obs)*100:.2f}% completed')           
        return ss
    return animation.FuncAnimation(fig, update, frames=np.arange(0,len(dat.z.obs)-1,fps), interval=1)

In [ ]:
ss =[]
ani = anim(dat,fps=1)
f = r'/ocean/mattmiller/MOAD/results/parcels/test/_1.gif'
FFwriter = animation.FFMpegWriter()
ani.save(f, writer = FFwriter)

In [ ]:
from IPython.display import Image
with open(f,'rb') as anim:
     display(Image(anim.read()))

In [ ]:
def anim(dat,fps=1):  
    fig, (ax1, ax2) = plt.subplots(1,2,figsize=(10,8))
    ax1.set_ylim(0,250)
    ax1.set_yticks([0, 10, 25, 50, 75, 100, 150, 250])
    ax1.invert_yaxis()
    ax1.xaxis.tick_top()
    ax1.xaxis.set_label_position('top')
    ax1.set_xlabel('Lon')
    ax1.set_ylabel('Depth (m)')
    ax2.set_ylim(0,250)
    ax2.invert_yaxis()
    ax2.set_yticks([0, 10, 25, 50, 75, 100, 150, 250]) # Set the tick positions and labels for the y-axis
    ax2.xaxis.tick_top()
    ax2.xaxis.set_label_position('top')
    ax2.set_xticks(np.arange(0, 1.1, 0.1)) # Set the tick positions and labels for the x-axis
    ax2.set_xlabel('Relative Particle Abundance') # Add labels and title
    ax2.set_ylabel('Depth (m)')
    plt.suptitle('Sink and Swim')

    bin_centers = [0,  10,  25,  50,  75, 100, 150] # Calculate the bin centers (for placing bins on the y-axis)
    binwidths = np.diff([0, 10, 25, 50, 75, 100, 150, 250]) # Define the binwidths
    vertical_dist_medians = [0.350311667, 0.239004743, 0.113561039, 0.025229733, 0.007112793, 0.002542321, 0.003152159]

    def update(frame):           
        global ss        
        for scat in ss:              
            scat.remove()            
        ss =[]        
        ss.append(ax1.scatter(dat.lon[:,frame],dat.z[:,frame],s=3,c ='tab:blue')) 
        tstamp = dat.time[0, frame].values.astype('timedelta64[ns]').astype(datetime)
        ss.append(ax1.text(x = 0.02, y = -0.03, s = tstamp, transform=ax1.transAxes))
        counts, bins = np.histogram(dat.z[:,frame], bins=[0, 10, 25, 50, 75, 100, 150, 250]) # Calculate the counts and bins for the histogram            
        relative_abundance = (counts/binwidths)/sum(counts/binwidths) # Calculate the relative particle abundance
        ss.append(ax2.barh(bin_centers, relative_abundance, label = 'particle dist', height=binwidths, align='edge', color='tab:blue'))                             
        ss.append(ax2.barh(bin_centers, vertical_dist_medians, label = 'reference dist', height=binwidths, align='edge', color='none', edgecolor ='r'))
        ss.append(ax2.text(x = 0.02, y = -0.03, s = tstamp, transform=ax2.transAxes))
        ss.append(ax2.legend())
        print(f'{frame/len(dat.z.obs)*100:.2f}% completed')           
        return ss
    return animation.FuncAnimation(fig, update, frames=np.arange(0,len(dat.z.obs)-1,fps), interval=1)

In [ ]:
ss =[]
ani = anim(dat,fps=1)
f = r'/ocean/mattmiller/MOAD/results/parcels/test/sink_and_swim.gif'
FFwriter = animation.FFMpegWriter()
ani.save(f, writer = FFwriter)

In [ ]:
from IPython.display import Image
with open(f,'rb') as anim:
     display(Image(anim.read()))

In [ ]:
len(dat.z.obs)-1